In [1]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-
"""
Mapa Interativo: Uso do Solo CLIP - Múltiplos Municípios
Classes de uso do solo classificadas com CLIP-ViT-B/32
"""

import pandas as pd
import geopandas as gpd
import folium
from folium import GeoJson
import json
import os

# ============================================================================
# 1. DEFINIÇÃO DE CORES E PALETAS
# ============================================================================

# Cores para classes de uso do solo (mesmas da referência)
class_colors = {
    'bare': '#bf8040',        # Marrom (solo exposto)
    'bush': '#009900',        # Verde médio
    'crop': '#ffdf99',        # Amarelo claro (agricultura)
    'grass': '#33ff33',       # Verde claro/lima
    'hduf': '#69000d',        # Vermelho escuro (alta densidade urbana)
    'industrial': '#e93529',  # Vermelho alaranjado (industrial)
    'lduf': '#c7171c',        # Vermelho médio (baixa densidade urbana)
    'mduf': '#a10e15',        # Vermelho médio-escuro (média densidade urbana)
    'tree': '#003300',        # Verde escuro
    'water': '#0b9fd5'        # Azul (água)
}

# Mapeamento de classes numéricas para nomes
class_to_name = {
    0: 'bare',
    1: 'bush',
    2: 'crop',
    3: 'grass',
    4: 'hduf',
    5: 'industrial',
    6: 'lduf',
    7: 'mduf',
    8: 'tree',
    9: 'water'
}

# Labels para exibição (ordem da referência)
class_labels = {
    'hduf': 'Developed: High Density',
    'mduf': 'Developed: Medium Density',
    'lduf': 'Developed: Low Density',
    'industrial': 'Developed: Industrial Areas',
    'bare': 'Bare Soil',
    'grass': 'Grass/Pasture',
    'bush': 'Shrub/Scrubs',
    'tree': 'Perennial Trees',
    'water': 'Perennial Water',
    'crop': 'Row Crops'
}

# ============================================================================
# 2. CONFIGURAÇÃO DOS MUNICÍPIOS
# ============================================================================

cities_paths = {
    'Londrina': '/Volumes/ssd_externo/UEL DOUTORADO 2022/Artigo CLIP GEO/Londrina/classificacaoLondrina.shp',
    'Cambe': '/Volumes/ssd_externo/UEL DOUTORADO 2022/Artigo CLIP GEO/Cambe/classificacaoCambe.shp',
    'Ibipora': '/Volumes/ssd_externo/UEL DOUTORADO 2022/Artigo CLIP GEO/Ibipora/classificacaoIbipora.shp',
    'Apucarana': '/Volumes/ssd_externo/UEL DOUTORADO 2022/Artigo CLIP GEO/Apucarana/classificacaoApucarana.shp',
    'Arapongas': '/Volumes/ssd_externo/UEL DOUTORADO 2022/Artigo CLIP GEO/Arapongas/classificacaoArapongas.shp',
    'Cambira': '/Volumes/ssd_externo/UEL DOUTORADO 2022/Artigo CLIP GEO/Cambira/classificacaoCambira.shp',
    'Jandaia': '/Volumes/ssd_externo/UEL DOUTORADO 2022/Artigo CLIP GEO/Jandaia/classificacaoJandaia.shp',
    'Mandaguari': '/Volumes/ssd_externo/UEL DOUTORADO 2022/Artigo CLIP GEO/Mandaguari/classificacaoMandaguari.shp',
    'Marialva': '/Volumes/ssd_externo/UEL DOUTORADO 2022/Artigo CLIP GEO/Marialva/classificacaoMarialva.shp',
    'Rolandia': '/Volumes/ssd_externo/UEL DOUTORADO 2022/Artigo CLIP GEO/Rolandia/classificacaoRolandia.shp',
    'Sarandi': '/Volumes/ssd_externo/UEL DOUTORADO 2022/Artigo CLIP GEO/Sarandi/classificacaoSarandi.shp',
    'Maringa': '/Volumes/ssd_externo/UEL DOUTORADO 2022/Artigo CLIP GEO/Maringa/classificacaoMaringa.shp'
}

# ============================================================================
# 3. FUNÇÃO PARA CALCULAR ÁREAS
# ============================================================================

def calculate_areas_from_gdf(gdf, city_name):
    """
    Calcula áreas de cobertura do solo por classe a partir de GeoDataFrame
    """
    # Converter para sistema de coordenadas projetado (UTM) para cálculo de área
    if gdf.crs != 'EPSG:29192':
        gdf_utm = gdf.to_crs('EPSG:29192')
    else:
        gdf_utm = gdf.copy()
    
    results = {}
    class_list = ['bare', 'bush', 'crop', 'grass', 'hduf', 'industrial', 'lduf', 'mduf', 'tree', 'water']
    
    for class_name in class_list:
        # Encontrar o número da classe
        class_num = [k for k, v in class_to_name.items() if v == class_name][0]
        
        # Filtrar polígonos da classe
        class_gdf = gdf_utm[gdf_utm['pred'] == class_num]
        
        # Calcular área total em m² e converter para km²
        if len(class_gdf) > 0:
            area_m2 = class_gdf.geometry.area.sum()
            area_km2 = area_m2 / 1e6
        else:
            area_km2 = 0.0
        
        results[f"area_{class_name}_km2_{city_name}"] = area_km2
    
    return results

# ============================================================================
# 4. CARREGAR DADOS E CALCULAR ÁREAS
# ============================================================================

print("Carregando shapefiles...")

city_gdfs = {}
city_results = {}
all_bounds = []

for city_name, shapefile_path in cities_paths.items():
    try:
        # Carregar shapefile
        gdf = gpd.read_file(shapefile_path)
        
        # Converter para EPSG:4326 para exibição no mapa
        if gdf.crs != 'EPSG:4326':
            gdf = gdf.to_crs('EPSG:4326')
        
        # Adicionar nome do município
        gdf['municipio'] = city_name
        
        # Adicionar nomes de classes e cores
        gdf['class_name'] = gdf['pred'].map(class_to_name)
        gdf['color'] = gdf['class_name'].map(class_colors)
        gdf['class_label'] = gdf['class_name'].map(class_labels)
        
        # Calcular áreas
        results = calculate_areas_from_gdf(gdf, city_name)
        city_results[city_name] = results
        
        # Armazenar GeoDataFrame
        city_gdfs[city_name] = gdf
        all_bounds.extend(gdf.total_bounds)
        
        print(f"✓ {city_name}: {len(gdf):,} polígonos")
        
    except Exception as e:
        print(f"✗ Erro ao carregar {city_name}: {e}")

# Calcular totais por município
class_list = ['bare', 'bush', 'crop', 'grass', 'hduf', 'industrial', 'lduf', 'mduf', 'tree', 'water']
city_totals = {}

for city_name in city_results.keys():
    total = sum([city_results[city_name][f"area_{cls}_km2_{city_name}"] for cls in class_list])
    city_totals[city_name] = total

# Criar tabela para exibição no console
print("\n" + "="*140)
print("LAND COVER AREAS BY MUNICIPALITY (CLIP Model)")
print("="*140)

table_data = []
for class_key in ['hduf', 'mduf', 'lduf', 'industrial', 'bare', 'grass', 'bush', 'tree', 'water', 'crop']:
    class_label = class_labels[class_key]
    row = {'Land Cover Class': class_label}
    
    for city_name in cities_paths.keys():
        if city_name in city_results:
            area_km2 = city_results[city_name][f"area_{class_key}_km2_{city_name}"]
            area_pct = (area_km2 / city_totals[city_name]) * 100 if city_totals[city_name] > 0 else 0
            row[city_name] = f"{area_km2:.2f} ({area_pct:.2f}%)"
    
    table_data.append(row)

# Adicionar linha de totais
totals_row = {'Land Cover Class': 'Totals (km²)'}
for city_name in cities_paths.keys():
    if city_name in city_totals:
        totals_row[city_name] = f"{city_totals[city_name]:.2f}"
table_data.append(totals_row)

df_table = pd.DataFrame(table_data)
print(df_table.to_string(index=False))
print("="*140 + "\n")

# ============================================================================
# 5. CRIAR MAPA INTERATIVO
# ============================================================================

print("Criando mapa interativo...")

# Calcular centro do mapa
if all_bounds:
    center_lat = (min(all_bounds[1::4]) + max(all_bounds[3::4])) / 2
    center_lon = (min(all_bounds[0::4]) + max(all_bounds[2::4])) / 2
else:
    center_lat, center_lon = -23.4, -51.5

# Criar mapa base com satélite
m = folium.Map(
    location=[center_lat, center_lon],
    zoom_start=10,
    tiles='https://mt1.google.com/vt/lyrs=s&x={x}&y={y}&z={z}',
    attr='Google Satellite'
)

# Adicionar cada cidade ao mapa
for city_name, gdf in city_gdfs.items():
    geojson_data = json.loads(gdf.to_json())
    
    city_group = folium.FeatureGroup(name=city_name, show=True)
    
    GeoJson(
        geojson_data,
        style_function=lambda feature: {
            'fillColor': feature['properties']['color'],
            'color': feature['properties']['color'],
            'weight': 0,
            'opacity': 0,
            'fillOpacity': 0.6
        },
        tooltip=folium.GeoJsonTooltip(
            fields=['municipio', 'class_label'],
            aliases=['', ''],
            labels=False
        )
    ).add_to(city_group)
    
    city_group.add_to(m)

print("✓ Camadas adicionadas ao mapa")

# ============================================================================
# 6. CRIAR LEGENDA (ESTILO REFERÊNCIA)
# ============================================================================

print("Criando legendas...")

legend_uso_html = """
<div style="position: fixed; 
           bottom: 10px; left: 10px; 
           background-color: white; border:2px solid grey; z-index:9999; 
           font-size:11px; padding: 15px; max-height: 90vh; max-width: 50vw;
           overflow-y: auto; overflow-x: auto;">
<p style="margin:0 0 10px 0; font-size:13px;"><b>Land Cover Areas by Municipality (CLIP Model)</b></p>
<table style="border-collapse: collapse; font-size:8px; width:100%;">
<thead>
    <tr style="background-color: #f0f0f0;">
        <th style="border: 1px solid #999; padding: 5px; text-align: left; font-weight: bold;">Land Cover Class</th>
"""

# Adicionar headers das cidades
for city_name in cities_paths.keys():
    legend_uso_html += f'<th style="border: 1px solid #999; padding: 5px; text-align: center; font-weight: bold;">{city_name}<br>km² (%)</th>\n'

legend_uso_html += """
    </tr>
</thead>
<tbody>
"""

# Mapeamento de classes para labels (ordem da referência)
table_classes = [
    ('hduf', 'Developed: High Density'),
    ('mduf', 'Developed: Medium Density'),
    ('lduf', 'Developed: Low Density'),
    ('industrial', 'Developed: Industrial Areas'),
    ('bare', 'Bare Soil'),
    ('grass', 'Grass/Pasture'),
    ('bush', 'Shrub/Scrubs'),
    ('tree', 'Perennial Trees'),
    ('water', 'Perennial Water'),
    ('crop', 'Row Crops')
]

# Adicionar linhas de dados
for class_key, class_label in table_classes:
    color = class_colors[class_key]
    text_color = 'white' if class_key in ['hduf', 'mduf', 'tree'] else 'black'
    
    legend_uso_html += f'<tr>\n'
    legend_uso_html += f'<td style="border: 1px solid #999; padding: 5px; background-color: {color}; color: {text_color}; font-weight: bold;">{class_label}</td>\n'
    
    for city_name in cities_paths.keys():
        if city_name in city_results:
            area_km2 = city_results[city_name][f"area_{class_key}_km2_{city_name}"]
            area_pct = (area_km2 / city_totals[city_name]) * 100 if city_totals[city_name] > 0 else 0
            legend_uso_html += f'<td style="border: 1px solid #999; padding: 5px; text-align: center;">{area_km2:.2f} ({area_pct:.2f}%)</td>\n'
    
    legend_uso_html += '</tr>\n'

# Adicionar linha de totais
legend_uso_html += '<tr style="background-color: #f0f0f0; font-weight: bold;">\n'
legend_uso_html += '<td style="border: 1px solid #999; padding: 6px;">Total Municipality</td>\n'

for city_name in cities_paths.keys():
    if city_name in city_totals:
        legend_uso_html += f'<td style="border: 1px solid #999; padding: 6px; text-align: center;">{city_totals[city_name]:.2f}</td>\n'

legend_uso_html += """
    </tr>
</tbody>
</table>
</div>
"""

# Adicionar legenda ao mapa
m.get_root().html.add_child(folium.Element(legend_uso_html))

print("✓ Legendas criadas")

# ============================================================================
# 7. SALVAR MAPA
# ============================================================================

output_path = "/Users/fjcosta/Documents/landCoverlandValue/landcover/landcoverCLIP_NP.html"
m.save(output_path)

# Verificar tamanho
file_size_mb = os.path.getsize(output_path) / (1024 * 1024)

print(f"\n{'='*70}")
print(f"✅ MAPA CRIADO COM SUCESSO!")
print(f"{'='*70}")
print(f"Arquivo: {output_path}")
print(f"\nEstatísticas:")
print(f"  • Municípios: {len(city_gdfs)}")
print(f"  • Polígonos totais: {sum(len(gdf) for gdf in city_gdfs.values()):,}")
print(f"  • Tamanho do arquivo: {file_size_mb:.2f} MB")
print(f"{'='*70}")

Carregando shapefiles...
✓ Londrina: 18,093 polígonos
✓ Cambe: 5,918 polígonos
✓ Ibipora: 3,902 polígonos
✓ Apucarana: 9,043 polígonos
✓ Arapongas: 10,671 polígonos
✓ Cambira: 238 polígonos
✓ Jandaia: 1,693 polígonos
✓ Mandaguari: 3,520 polígonos
✓ Marialva: 2,297 polígonos
✓ Rolandia: 3,771 polígonos
✓ Sarandi: 2,342 polígonos
✓ Maringa: 11,481 polígonos

LAND COVER AREAS BY MUNICIPALITY (CLIP Model)
           Land Cover Class       Londrina          Cambe        Ibipora      Apucarana      Arapongas       Cambira       Jandaia     Mandaguari      Marialva       Rolandia        Sarandi        Maringa
    Developed: High Density   4.22 (1.95%)   0.16 (0.22%)   0.00 (0.00%)   0.68 (0.63%)   0.35 (0.27%)  0.00 (0.00%)  0.15 (0.77%)   0.00 (0.00%)  0.00 (0.00%)   0.02 (0.05%)   0.01 (0.04%)   2.43 (1.77%)
  Developed: Medium Density 33.75 (15.59%)  9.68 (13.67%)   1.78 (3.82%)   8.44 (7.83%)   7.60 (5.96%) 0.63 (22.27%) 2.57 (12.76%)   1.94 (4.63%)  1.21 (4.40%)  4.80 (10.63%)   2.32 (8.